In [2]:
import pandas as pd
import numpy as np
import random
import pickle

In [81]:
vb = pd.read_csv("../data/vb_data_3_numZone.csv")
print(len(vb), 'lines loaded')

146050 lines loaded


In [5]:
game_count = len(vb.GameID.unique())
print(game_count)

204


In [21]:
vb_shuffle = pd.DataFrame(columns=vb.columns)
games = list(vb.GameID.unique())
random.shuffle(games)
for game in games:
    game_data = vb.loc[vb.GameID==game]
    vb_shuffle = pd.concat([vb_shuffle, game_data], ignore_index=True)

In [22]:
vb_shuffle.head()

,Season,GameID,PlayerTeam,PlayerName,RewardValue,RewardDistance,SetNumber,ScoreMax,ScoreDiff,ActionHome9,...,ActionOutcome1,ActionHome0,ActionAway0,ActionType0,ActionStartX0,ActionStartY0,ActionEndX0,ActionEndY0,ActionSpeed0,ActionOutcome0
0,2019/2020,Canada West MVB<19>: Trinity Western - UBCO,Trinity Western,Brodie Hofer,-1,4,1,0,0,0,...,0,1,0,S,4.50,7.50,4.50,7.50,1,-
1,2019/2020,Canada West MVB<19>: Trinity Western - UBCO,UBCO,Denham O'Reilly,-1,3,1,0,0,0,...,-,0,1,R,4.50,7.50,6.75,0.75,1,+
2,2019/2020,Canada West MVB<19>: Trinity Western - UBCO,UBCO,Jakob Hamilton,-1,2,1,0,0,0,...,+,0,1,E,6.75,0.75,8.25,0.75,1,#
3,2019/2020,Canada West MVB<19>: Trinity Western - UBCO,UBCO,Cam Vanderveen,-1,1,1,0,0,0,...,#,0,1,A,7.50,1.50,4.50,1.50,1,#
4,2019/2020,Canada West MVB<19>: Trinity Western - UBCO,Trinity Western,Derek Epp,-1,0,1,0,0,0,...,#,1,0,B,1.50,1.50,1.50,1.50,-1,-


In [24]:
from keras.models import Sequential
from keras.layers import Dense

In [79]:
X = vb.drop(['Season', 'GameID', 'PlayerTeam', 'PlayerName', 'RewardDistance', 'RewardValue'], axis=1)


cols = [col for col in list(X.columns) if X[col].dtype == 'object']
X = pd.get_dummies(data=X, columns=cols)
y = vb.RewardValue

print(len(X.columns), 'columns in numerical dataframe after dummies')

NameError: name 'vb' is not defined

In [58]:
model = Sequential()
model.add(Dense(1024, input_dim=204, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='linear'))

In [59]:
model.compile(loss='mse', optimizer='adam')

In [64]:
model.fit(X, y, epochs=5, batch_size=32, shuffle=True, validation_split=0.2, verbose=1)

Train on 116840 samples, validate on 29210 samples
Epoch 1/5
116840/116840 [==============================] - 36s 312us/step - loss: 0.5842 - val_loss: 0.6405
Epoch 2/5
116840/116840 [==============================] - 36s 312us/step - loss: 0.5824 - val_loss: 0.6414
Epoch 3/5
116840/116840 [==============================] - 37s 313us/step - loss: 0.5817 - val_loss: 0.6415
Epoch 4/5
116840/116840 [==============================] - 37s 313us/step - loss: 0.5801 - val_loss: 0.6450
Epoch 5/5
116840/116840 [==============================] - 37s 313us/step - loss: 0.5783 - val_loss: 0.6433


In [62]:
y_pred = model.predict(X)

In [63]:
from sklearn import metrics
metrics.mean_squared_error(vb.RewardValue, y_pred)

0.5918273591603079

In [7]:
save_dir = 'C:/projects/msc-project/output_retry_8'
state_data = pickle.load(open(save_dir + '/state_data.pkl', 'rb'))
reward_data = pickle.load(open(save_dir + '/reward_data.pkl', 'rb'))
reward_targets = pickle.load(open(save_dir + '/reward_targets.pkl', 'rb'))
trace_lengths = pickle.load(open(save_dir + '/trace_lengths.pkl', 'rb'))
episode_ids = pickle.load(open(save_dir + '/episode_ids.pkl', 'rb'))

In [8]:
episode_ids

array([1.0000e+00, 1.0000e+00, 1.0000e+00, ..., 3.4371e+04, 3.4371e+04,
       3.4371e+04])

In [63]:
def shuffle_data(state_data, reward_data, trace_lengths, episode_ids):
    
    n_samples = len(reward_data)
    state_data_new = np.zeros([n_samples, 10, 41])
    trace_lengths_new = np.zeros([n_samples])
    reward_data_new = np.zeros([n_samples])
    episode_ids_new = np.zeros([n_samples])

    shuffled_episode_ids = list(range(1, (int)(np.max(episode_ids))+1))
    random.shuffle(shuffled_episode_ids)

    idx = 0
    for ep_id in shuffled_episode_ids:
        cond = np.where(episode_ids == ep_id)
        i1 = cond[0][0]
        i2 = cond[0][-1]+1
        n = len(cond[0])
        state_data_new[idx:idx+n,:,:] = state_data[i1:i2,:,:]
        reward_data_new[idx:idx+n] = reward_data[i1:i2]
        trace_lengths_new[idx:idx+n] = trace_lengths[i1:i2]
        episode_ids_new[idx:idx+n] = episode_ids[i1:i2]
        idx += n

    return state_data_new, reward_data_new, trace_lengths_new, episode_ids_new

In [75]:
state_data1, reward_data1, trace_lengths1, episode_ids1 = shuffle_data(state_data, reward_data, trace_lengths, episode_ids)
state_data2, reward_data2, trace_lengths2, episode_ids2 = shuffle_data(state_data1, reward_data1, trace_lengths1, episode_ids1)

In [76]:
episode_ids2[0:100]

array([28364., 28364., 28364., 28364., 28364., 19905., 19935., 19935.,
       19935., 19935., 33350., 33350.,  8199.,  8199.,  8199.,  8199.,
        8199.,  8199.,    40.,    40.,    40.,    40.,    40., 21823.,
       21823., 21823., 21823., 33980., 33980., 33980., 33980., 33980.,
       10738., 10738., 16749., 16749., 16749., 16749., 16749., 18782.,
       18782., 18782., 18782., 31987., 31987., 31987., 31987., 31987.,
        6265.,  6265.,  6265.,  6265.,  6265.,  6265.,  6265., 23572.,
       23572., 23572., 23572., 23572., 23572., 13391., 13391., 13391.,
       13391., 13391.,  6825.,  6825.,  6825.,  6825.,  6825.,  6825.,
        6825., 18657., 18657., 18657., 10898., 10898., 10898., 10898.,
       24054., 24054., 24054., 24054., 24054., 22333., 22333., 22333.,
       22333., 22333., 22333., 13767., 13767., 13767., 13767., 23321.,
       23321., 23321., 23321., 23321.])

In [94]:
s0 = reward_data[np.where(episode_ids==9454)[0]]
s1 = reward_data1[np.where(episode_ids1==9454)[0]]
s2 = reward_data2[np.where(episode_ids2==9454)[0]]
print(np.sum(s1-s2))
print(s0)
print(s1)
print(s2)

0.0
[ 0. -1.]
[ 0. -1.]
[ 0. -1.]


In [105]:
print(reward_data[np.where(episode_ids==9453)[0]])

[-1.]


In [106]:
vb.iloc[np.where(episode_ids==9453)[0]]

,Season,GameID,PlayerTeam,PlayerName,RewardValue,RewardDistance,SetNumber,ScoreMax,ScoreDiff,ActionHome9,...,ActionOutcome1,ActionHome0,ActionAway0,ActionType0,ActionStartX0,ActionStartY0,ActionEndX0,ActionEndY0,ActionSpeed0,ActionOutcome0
40154,2018/2019,Canada West MVB<24>: MacEwan - Calgary,MacEwan,Max Vriend,-1,0,3,5,-1,0,...,0,1,0,S,7.5,7.5,1.5,7.5,-1,=
